In [ ]:
from itertools import product
from random import random, randint, seed, choice
from functools import reduce
from copy import copy
import numpy as np
from scipy import sparse

In [ ]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [ ]:
def fitness(state, problem_size, sets):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_size)]),
        )
    )
    return valid, -cost

In [ ]:
def tweak(state, problem_size):
    new_state = copy(state)
    index = randint(0, problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state

### First-Improvement Hill Climber

In [ ]:
def fi_hill_climber(sets, initial_state, steps, problem_size):
    counter = 0
    current_state = initial_state

    for _ in range(steps):
        new_state = tweak(current_state, problem_size)
        counter += 2
        if fitness(new_state, problem_size, sets) >= fitness(current_state, problem_size, sets):
            current_state = new_state

    return current_state, counter

### Steepest-Step Hill Climber

In [ ]:
def ss_hill_climber(sets, initial_state, steps, problem_size, n):
    counter = 0
    current_state = initial_state

    for _ in range(steps):
        new_state1 = tweak(current_state, problem_size)
        for _ in range(n - 1):
            new_state2 = tweak(current_state, problem_size)
            counter += 2
            if fitness(new_state1, problem_size, sets) >= fitness(new_state2, problem_size, sets):
                new_state1 = new_state2
        
        counter += 2
        if fitness(new_state1, problem_size, sets) >= fitness(current_state, problem_size, sets):
            current_state = new_state1

    return current_state, counter

### Iterated Local Search

In [ ]:
def iterated_local_search(sets, initial_state, steps, problem_size, n_restart):
    counter = 0
    for _ in range(n_restart):
        solution, n_call = fi_hill_climber(sets, initial_state, steps, problem_size)
        initial_state = solution
        counter += n_call

    return solution, counter

### Tabu Search 

In [ ]:
def tabu_search(sets, initial_state, steps, problem_size, n):
    counter = 0
    tabu_list = []
    best_state = initial_state
    best_value = fitness(initial_state, problem_size, sets)
    counter += 1

    for _ in range(steps):
        tmp = [tweak(best_state, problem_size) for _ in range(n)]
        candidates = [sol for sol in tmp if sol not in tabu_list]

        if not candidates:
            continue

        best_candidate = candidates[0]
        best_candidate_value = fitness(best_candidate, problem_size, sets)
        counter += 1

        for c in candidates:
            counter += 1
            candidate_value = fitness(c, problem_size, sets)
            
            if candidate_value > best_candidate_value:
                best_candidate = c
                best_candidate_value = candidate_value

            tabu_list.append(c)

        if best_candidate_value > best_value:
            best_state = best_candidate
            best_value = best_candidate_value
        
    return best_state, counter




    
    

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [ ]:
num_points = [100, 1000, 5000]
density = [0.3, 0.7]
steps = 1000
n_restart = 10
n = 20

In [ ]:
for d in density:
    for problem_size in num_points:
        print('Num_points = {0}\tDensity = {1}'.format(problem_size, d))
        sets = make_set_covering_problem(problem_size, problem_size, d)
        initial_state = [choice([True, False]) for _ in range(problem_size)]

        #First-Improvement Hill Climber
        solution, n_calls = fi_hill_climber(sets, initial_state, steps, problem_size)
        print('\tFirst-Improvement Hill Climber --> Best solution = {0}\tFitness calls = {1}'.format(fitness(solution, problem_size, sets), n_calls))

        #Steepest-Step Hill Climber
        solution, n_calls = ss_hill_climber(sets, initial_state, steps, problem_size, n)
        print('\tSteepest-Step Hill Climber --> Best solution = {0}\tFitness calls = {1}'.format(fitness(solution, problem_size, sets), n_calls))

        #Iterated Local Search
        solution, n_calls = iterated_local_search(sets, initial_state, steps, problem_size, n_restart)
        print('\tIterated Local Search --> Best solution = {0}\tFitness calls = {1}'.format(fitness(solution, problem_size, sets), n_calls))

        #Tabu Search
        solution, n_calls = tabu_search(sets, initial_state, steps, problem_size, n)
        print('\tTabu Search --> Best solution = {0}\tFitness calls = {1}'.format(fitness(solution, problem_size, sets), n_calls))
        print('\n')